In [1]:
import numpy as np
import copy
from multiprocessing import Pool
import multiprocessing

In [2]:
def init_pi(N_a,N_s):
    """
    function to generate policy,
    inputs:
        N_a - number of actions;
        N_s - number of states;
    outputs:
        pi(a|s) - np.array of shape N_a x N_s
    """
    np.random.seed(1453)
    Pi_matr = np.random.uniform(0.0,1.0,(N_a,N_s))
    norm_coef = Pi_matr.sum(axis=0)
    Pi_matr = Pi_matr / norm_coef.reshape((1,N_s))
    #check if stochastic
    print(Pi_matr.sum(axis=0))
    return Pi_matr

In [3]:
def generate_dynamics(N_a,N_s,b):
    """
    function to generate transition probabilities,
    inputs:
        N_a - number of actions;
        N_s - number of states;
        b - branching number
    outputs:
        pi(s'|s,a) - np.array of shape N_s x N_s x N_a
    """
    np.random.seed(1812)
    inds_nonzero = np.zeros((N_s,N_a,b),dtype = int)
    for i in range(N_s):
        for j in range(N_a):
            inds_nonzero[i,j] = np.random.choice(N_s, size=b, replace=False)
    Pi_matr = np.zeros((N_s,N_s,N_a),dtype=float)
    for i in range(N_s):
        for j in range(N_a):
            Pi_matr[inds_nonzero[i,j],i,j] = np.random.uniform(0.0,1.0,b)
    norm_coef = Pi_matr.sum(axis=0)
    Pi_matr = Pi_matr / norm_coef.reshape((1,N_s,N_a))
    print(Pi_matr.sum(axis=0))
    return Pi_matr,inds_nonzero

In [4]:
def state_transitions(P,pi):
    """
    function to generate transition probabilities,
    inputs:
        P(s'|s,a) - np.array of shape N_s x N_s x N_a, transition probabilities;
        pi(a|s) - np.array of shape N_a x N_s, policy;
    outputs:
        p(s'|s) - transition probability matrix of shape (N_s,N_s)
    """
    np.random.seed(1812)
    P_s = np.zeros((N_s,N_s),dtype = float)
    for i in range(N_s):
        for j in range(N_s):
            P_s[i,j] = np.dot(P[i,j,:],pi[:,j])
    return P_s 

In [5]:
def init_rewards(N_a,N_s):
    """
    function to generate rewards,
    inputs:
        N_a - number of actions;
        N_s - number of states;
    outputs:
        R(a,s) - np.array of rewards (shape N_a x N_s)  
    """
    np.random.seed(1821)
    R = Pi_matr = np.random.uniform(0.0,2.0,(N_a,N_s))
    return R

In [6]:
#global constants
N_a = 10
N_s = 50
#gamma = 0.99
gamma = 0.9
b = 2

In [7]:
#init policy matrix
Policy = init_pi(N_a,N_s)
#init transition matrix
P,Inds_nz = generate_dynamics(N_a,N_s,b)
#init rewards
R = init_rewards(N_a,N_s)
#init state transition matrix
S_trans = state_transitions(P,Policy)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1

### Solve system to find $\theta^*$ (i.e. true $V_{\pi}(s)$)

In [8]:
#system matrix
A = np.eye(N_s) - gamma*(S_trans.T)
#right hand side
b = np.sum(Policy*R,axis=0)
theta_star = np.linalg.inv(A) @ b
print(theta_star)

[9.84953413 9.52818882 9.10077358 9.20650432 9.46348305 9.53415339
 9.60132343 9.58224446 9.02248147 9.55440038 9.61889827 9.51152889
 9.24885002 9.77558082 9.80059676 9.89326659 9.40835507 9.31492265
 9.19517124 9.50103756 9.20002613 9.22017523 9.09961668 9.38826035
 9.25805059 9.07006076 9.32369281 9.5107984  9.51615943 9.50436866
 9.38564581 9.64467251 9.71316469 9.33079435 9.09258728 9.66603574
 9.11930009 9.36024236 9.35390133 9.24264092 9.57049198 9.25693893
 9.35382775 9.29097659 9.04836883 9.14591195 9.4543717  9.56269105
 9.68137285 9.68337974]


### Find stationary distribution over state space

In [9]:
#note that in my notations they are usual (right) eigenvalues
eigvals, eigfuncs = np.linalg.eig(S_trans)
print(eigvals)
pi_states = -np.real(eigfuncs[:,0])
pi_states = pi_states/np.sum(pi_states)
#pi_states - statinary distribution over states
print(pi_states)
print(np.sum(pi_states))

[ 1.        +0.j         -0.26430692+0.12903984j -0.26430692-0.12903984j
 -0.28404277+0.03470012j -0.28404277-0.03470012j -0.13168546+0.2269733j
 -0.13168546-0.2269733j   0.24202435+0.09985377j  0.24202435-0.09985377j
  0.13218168+0.22116056j  0.13218168-0.22116056j  0.0566535 +0.23111173j
  0.0566535 -0.23111173j  0.1641442 +0.1417069j   0.1641442 -0.1417069j
  0.21104999+0.j          0.19396104+0.06409327j  0.19396104-0.06409327j
  0.18542299+0.j          0.11909186+0.16312541j  0.11909186-0.16312541j
 -0.20534831+0.10750108j -0.20534831-0.10750108j -0.00321329+0.21593335j
 -0.00321329-0.21593335j -0.06451847+0.20039498j -0.06451847-0.20039498j
 -0.07494813+0.18662476j -0.07494813-0.18662476j  0.05846562+0.16833902j
  0.05846562-0.16833902j -0.20275598+0.j         -0.14950415+0.07123611j
 -0.14950415-0.07123611j -0.16728513+0.j          0.01340965+0.15919804j
  0.01340965-0.15919804j  0.12790264+0.j          0.07303261+0.09720718j
  0.07303261-0.09720718j -0.03112849+0.09440118j -0.0

### Find $\theta^*$ by another approach

In [10]:
A_star = np.zeros((N_s,N_s),dtype=float)
for i in range(N_s):
    for j in range(N_s):
        A_star[i,j] = pi_states[i]*S_trans[j,i]
A_star = np.diag(pi_states) - gamma*A_star
b_star = b*pi_states
theta_star_new = np.linalg.inv(A_star) @ b_star
print("error between to True solutions: ",np.linalg.norm(theta_star-theta_star_new))

error between to True solutions:  1.7852165294699933e-14


### Run TD$(0)$

In [11]:
N_iters = 2*10**6
v0 = np.zeros(N_s,dtype=float)
s0 = np.random.choice(N_s)
#step size
alpha_0 = 1.0
alpha = np.zeros(N_iters,dtype = float)
N_0 = 10**3
powers = np.array([0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9],dtype=float)
alpha = np.zeros((len(powers),N_iters),dtype=float)
for j in range(len(powers)):
    for i in range(N_iters):
        alpha[j][i] = 100.0/(N_0+i)**(powers[j])

In [12]:
print(Policy.shape)
print(Policy.sum(axis=0))

(10, 50)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [13]:
def main_loop(j,alpha,s0):
    V_funcs = np.zeros((N_iters,N_s))
    J_0 = np.zeros((N_iters,N_s))
    J_1 = np.zeros((N_iters,N_s))
    Transient = np.zeros((N_iters,N_s))
    V = np.zeros(N_s,dtype=float)
    J0_cur = np.zeros(N_s,dtype=float)
    J1_cur = np.zeros(N_s,dtype=float)
    Transient_cur = v0 - theta_star
    A_tilde = np.zeros((N_s,N_s),dtype=float)
    ###Main loop
    for N in range(N_iters):
        #sample action
        a = np.random.choice(N_a, 1, replace=True, p=Policy[:,s0])
        a=a[0]
        #sample next state
        s = np.random.choice(Inds_nz[s0,a], 1, replace=True, p=P[Inds_nz[s0,a],s0,a])
        s=s[0]
        #calculate J0
        eps = np.zeros(N_s,dtype=float)
        eps[s0] = R[a,s0] + gamma*theta_star[s]-theta_star[s0]
        eps_TD = R[a,s0] + gamma*V[s]-V[s0]
        #calculate J1
        A_tilde[s0,s0] = 1.0
        A_tilde[s0,s] = -gamma
        J1_cur = (np.eye(N_s) - alpha[j][N]*A_star)@J1_cur - alpha[j][N]*(A_tilde-A_star)@J0_cur
        #calculate transient term
        Transient_cur = (np.eye(N_s) - alpha[j][N]*A_tilde)@Transient_cur
        #calculate J0
        J0_cur = (np.eye(N_s) - alpha[j][N]*A_star)@J0_cur + alpha[j][N]*eps
        #TD update
        V[s0] = V[s0] + alpha[j][N]*eps_TD
        #save value function
        V_funcs[N] = V
        #save J_0
        J_0[N] = J0_cur
        #save J_1
        J_1[N] = J1_cur
        #save transient term
        Transient[N] = Transient_cur
        #vanish A_tilde
        A_tilde[s0,s0] = 0.0
        A_tilde[s0,s] = 0.0
        #update current state
        s0 = s
    return np.asarray([V_funcs,J_0,J_1,Transient])
    

In [ ]:
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res_indep = trav.starmap(main_loop, [(j,alpha,s0) for j in range (len(powers))])
trav.close()

In [ ]:
res_indep = np.asarray(res_indep)
print(res_indep.shape)

In [ ]:
norms = np.zeros((len(powers),N_iters),dtype=float)
norms_J0_rem = np.zeros((len(powers),N_iters),dtype=float)
norms_J1_rem = np.zeros((len(powers),N_iters),dtype=float)
norms_transient = np.zeros((len(powers),N_iters),dtype=float)

norms_J0 = np.zeros((len(powers),N_iters),dtype=float)
norms_J1 = np.zeros((len(powers),N_iters),dtype=float)
for j in range(len(powers)):
    for i in range(N_iters):
        norms[j][i] = np.linalg.norm(res_indep[j,0,i,:]-theta_star)
        norms_J0_rem[j][i] = np.linalg.norm(res_indep[j,0,i,:] - res_indep[j,1,i,:]-theta_star)
        norms_J1_rem[j][i] = np.linalg.norm(res_indep[j,0,i,:] - res_indep[j,1,i,:]-res_indep[j,2,i,:]-theta_star)
        norms_transient[j][i] = np.linalg.norm(res_indep[j,3,i,:])
        norms_J0[j][i] = np.linalg.norm(res_indep[j,1,i,:])
        norms_J1[j][i] = np.linalg.norm(res_indep[j,2,i,:])

### Save results

### Plot graphics

In [ ]:
import matplotlib.pyplot as plt
N_start = 0
j=3
plt.figure(figsize=(12,8)) 
plt.plot(np.arange(N_start,N_iters), norms[j][N_start:], color='r' ,label='MSE error') 
plt.plot(np.arange(N_start,N_iters), norms_J0_rem[j][N_start:], color='g' ,label='MSE error without J_0') 
plt.plot(np.arange(N_start,N_iters), norms_J1_rem[j][N_start:], color='b' ,label='MSE error without J_0, J_1')
plt.xlabel('iteration number',fontsize = 18)
#plt.ylabel('cost',fontsize = 18) 
#plt.title('VR cost for MDCV, Gaussian distribution, quadratic target',fontsize = 20)
plt.yscale('log')
plt.legend() 
plt.show()

In [ ]:
#initialize policy
V = copy.deepcopy(v0)
J0_cur = np.zeros(N_s,dtype=float)
J1_cur = np.zeros(N_s,dtype=float)
Transient_cur = v0 - theta_star
###Main loop
for N in range(N_iters):
    #sample action
    a = np.random.choice(N_a, 1, replace=True, p=Policy[:,s0])
    a=a[0]
    #sample next state
    s = np.random.choice(Inds_nz[s0,a], 1, replace=True, p=P[Inds_nz[s0,a],s0,a])
    s=s[0]
    #calculate J0
    eps = np.zeros(N_s,dtype=float)
    eps[s0] = R[a,s0] + gamma*theta_star[s]-theta_star[s0]
    eps_TD = R[a,s0] + gamma*V[s]-V[s0]
    #calculate J1
    A_tilde = np.zeros((N_s,N_s),dtype=float)
    A_tilde[s0,s0] = 1.0
    A_tilde[s0,s] = -gamma
    J1_cur = (np.eye(N_s) - alpha[N]*A_star)@J1_cur - alpha[N]*(A_tilde-A_star)@J0_cur
    #calculate transient term
    Transient_cur = (np.eye(N_s) - alpha[N]*A_tilde)@Transient_cur
    #calculate J0
    J0_cur = (np.eye(N_s) - alpha[N]*A_star)@J0_cur + alpha[N]*eps
    #TD update
    V[s0] = V[s0] + alpha[N]*eps_TD
    #save value function
    V_funcs[N] = V
    #save J_0
    J_0[N] = J0_cur
    #save J_1
    J_1[N] = J1_cur
    #save transient term
    Transient[N] = Transient_cur
    #update current state
    s0 = s

In [ ]:
norms = np.zeros(N_iters)
norms_J0_rem = np.zeros(N_iters)
norms_J1_rem = np.zeros(N_iters)
norms_transient = np.zeros(N_iters)

norms_J0 = np.zeros(N_iters)
norms_J1 = np.zeros(N_iters)
for i in range(N_iters):
    norms[i] = np.linalg.norm(V_funcs[i,:]-theta_star)
    norms_J0_rem[i] = np.linalg.norm(V_funcs[i,:] - J_0[i,:]-theta_star)
    norms_J1_rem[i] = np.linalg.norm(V_funcs[i,:] - J_0[i,:]-J_1[i,:]-theta_star)
    norms_transient[i] = np.linalg.norm(Transient[i])
    norms_J0[i] = np.linalg.norm(J_0[i,:])
    norms_J1[i] = np.linalg.norm(J_1[i,:])

In [ ]:
import matplotlib.pyplot as plt
N_start = 5*10**4
plt.figure(figsize=(12,8)) 
plt.plot(np.arange(N_start,N_iters), norms[N_start:], color='r' ,label='MSE error') 
plt.plot(np.arange(N_start,N_iters), norms_J0_rem[N_start:], color='g' ,label='MSE error without J_0') 
plt.plot(np.arange(N_start,N_iters), norms_J1_rem[N_start:], color='b' ,label='MSE error without J_0, J_1')
plt.xlabel('iteration number',fontsize = 18)
#plt.ylabel('cost',fontsize = 18) 
#plt.title('VR cost for MDCV, Gaussian distribution, quadratic target',fontsize = 20)
#plt.yscale('log')
plt.legend() 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
N_start = 1
plt.figure(figsize=(12,8)) 
plt.plot(np.arange(N_start,N_iters), norms_transient[N_start:], color='r' ,label='Norm of transient term') 
plt.plot(np.arange(N_start,N_iters), norms_J0[N_start:], color='g' ,label='Norm of J_0') 
plt.plot(np.arange(N_start,N_iters), norms_J1[N_start:], color='b' ,label='Norm of J_1') 
plt.xlabel('iteration number',fontsize = 18)
#plt.ylabel('cost',fontsize = 18) 
#plt.title('VR cost for MDCV, Gaussian distribution, quadratic target',fontsize = 20)
plt.yscale('log')
plt.legend() 
plt.show()